<a href="https://colab.research.google.com/github/manithamanoj/Amazon_Vine_Analysis/blob/main/Amazon_vine_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 https://developer.download.nvidia.com/c

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
#!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").getOrCreate()

In [7]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [8]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [9]:
df = df.dropna()
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [10]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [11]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|                Y|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|                Y|
|R2S79GCF6J89OA|          3|            0|          0|   N|     

In [12]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
from pyspark.sql.functions import col
total_votes_df = vine_df.filter(col("total_votes") >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
|R2P76PJFUGXBHO|          1|            6|         45|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|     

In [13]:
percent_votes_df = total_votes_df.withColumn('percent_votes',col('helpful_votes')/col('total_votes')).alias('percent_votes').filter(col("percent_votes") >= 0.5)
percent_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|            0.9375|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|0.9807692307692307|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|0.8055555555555556|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|0.9866666666666667|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|0.8846153846153846|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|0.9251700680272109|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|0.9047619047619048|
|R1P0LEEJHH09L3|          5|  

In [14]:
vine_paid_df=percent_votes_df.filter(percent_votes_df['vine']== 'Y')
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R6U9701C3BGO6|          3|          139|        147|   Y|                N|0.9455782312925171|
|R1XK3ALB45D7N4|          5|           33|         34|   Y|                N|0.9705882352941176|
|R1IZCSTLX81D6C|          5|           31|         33|   Y|                N|0.9393939393939394|
|R2C8NC8EQLH4JF|          3|           45|         48|   Y|                N|            0.9375|
|R1JJ1YOJMOML1P|          5|           18|         21|   Y|                N|0.8571428571428571|
|R3TKG664L9MTXJ|          4|          164|        175|   Y|                N|0.9371428571428572|
|R2E942L5EX73FP|          5|           29|         33|   Y|                N|0.8787878787878788|
|R3CGRT9HYB3LI7|          5|  

In [15]:

vine_nonpaid_df=percent_votes_df.filter(percent_votes_df['vine']== 'N')
vine_nonpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|            0.9375|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|0.9807692307692307|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|0.8055555555555556|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|0.9866666666666667|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|0.8846153846153846|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|0.9251700680272109|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|0.9047619047619048|
|R1P0LEEJHH09L3|          5|  

In [16]:
from pyspark.sql.functions import count
total_votes_df.select(count("vine")).show()

+-----------+
|count(vine)|
+-----------+
|      46767|
+-----------+



In [17]:
from pyspark.sql.functions import col, count

totalreviews=total_votes_df.count()
totalreviews

46767

In [18]:
totalpaidreviews=vine_paid_df.count()
totalpaidreviews

33

In [19]:
percentofpaidreviews=(totalpaidreviews/totalreviews)*100
percentofpaidreviews

0.07056257617550837

In [20]:
totalnonpaidreviews=vine_nonpaid_df.count()
totalnonpaidreviews

45246

In [21]:
percentofNonpaidreviews=(totalnonpaidreviews/totalreviews)*100
percentofNonpaidreviews

96.7477067162743

In [22]:
total5Starreviews=total_votes_df.filter(total_votes_df["star_rating"]=='5').count()
total5Starreviews

23897

In [23]:

vinePaid5starReviews=vine_paid_df.filter(vine_paid_df["star_rating"]=='5').count()
vinePaid5starReviews

15

In [24]:
vineNonPaid5starReviews=vine_nonpaid_df.filter(vine_nonpaid_df["star_rating"]=='5').count()
vineNonPaid5starReviews

23639

In [25]:
percentOfPaid5star=(vinePaid5starReviews/total5Starreviews)*100
percentOfPaid5star

0.06276938527848684

In [26]:
percentOfNonPaid5star=(vineNonPaid5starReviews/total5Starreviews)*100
percentOfNonPaid5star

98.92036657321003

In [27]:
percentOfPaid5startoTotal=(vinePaid5starReviews/totalreviews)*100
percentOfPaid5startoTotal

0.032073898261594715

In [28]:
percentOfNonPaid5startoTotal=(vineNonPaid5starReviews/totalreviews)*100
percentOfNonPaid5startoTotal

50.54632540038916

In [29]:

percentOfPaid5startoTotaltoPaid=(vinePaid5starReviews/totalpaidreviews)*100
percentOfPaid5startoTotaltoPaid

45.45454545454545

In [30]:
percentOfNonPaid5startoTotaltoNonPaid=(vineNonPaid5starReviews/totalnonpaidreviews)*100
percentOfNonPaid5startoTotaltoNonPaid

52.245502364849926

In [32]:
# Create DataFrame manually
dataframe = spark.createDataFrame([
                                    ('Y',vinePaid5starReviews,round(percentOfPaid5startoTotaltoPaid,2), round(percentOfPaid5star,2),round(percentOfPaid5startoTotal,2)),
                                    ('N',vineNonPaid5starReviews,round(percentOfNonPaid5startoTotaltoNonPaid,2), round(percentOfNonPaid5star,2),round(percentOfNonPaid5startoTotal,2)),
                                   
  ], ["Vine","Total 5star reviews","%ofPaid5star to TotalPaid& ,% of NonPaid5star to TotalNonPaid", "Percent of 5star reviews","Percent of 5star reviews to Total"])
dataframe.show()


+----+-------------------+-------------------------------------------------------------+------------------------+---------------------------------+
|Vine|Total 5star reviews|%ofPaid5star to TotalPaid& ,% of NonPaid5star to TotalNonPaid|Percent of 5star reviews|Percent of 5star reviews to Total|
+----+-------------------+-------------------------------------------------------------+------------------------+---------------------------------+
|   Y|                 15|                                                        45.45|                    0.06|                             0.03|
|   N|              23639|                                                        52.25|                   98.92|                            50.55|
+----+-------------------+-------------------------------------------------------------+------------------------+---------------------------------+

